# Optimized Budget Allocation for Maximizing Leads: A Campaign Performance Enhancement Model

---



---



In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 74.3 MB/s eta 0:00:00


In [23]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus
import pandas as pd

# Load campaign data from Excel
df = pd.read_excel("Campaign_Data_Valid_Updated.xlsx")

# Extract data
campaigns = df["Campaign Name"].tolist()
actual_leads = dict(zip(campaigns, df["Generated Accounts"].tolist()))
target_leads = dict(zip(campaigns, df["Target Accounts"].tolist()))
current_spend = dict(zip(campaigns, df["Media Spent ($)"].tolist()))
budget = dict(zip(campaigns, df["Campaign Budget ($)"].tolist()))
duration = dict(zip(campaigns, df["Campaign Duration (months)"].tolist()))  # New duration variable

# Define Optimization Model
model = LpProblem("Campaign_Spend_Optimization", LpMinimize)

# Compute weighted importance based on duration
total_duration = sum(duration.values())
weights = {c: duration[c] / total_duration for c in campaigns}

# Decision Variables (Optimized budget for each campaign)
x = {c: LpVariable(f"x_{c}", 0, budget[c]) for c in campaigns}  # Optimized budget
optimized_leads = {c: LpVariable(f"leads_{c}", actual_leads[c] + 1, None, cat='Integer') for c in campaigns}  # Ensure leads are strictly greater than actual leads and remain natural numbers

# Auxiliary variables for absolute deviation
deviation = {c: LpVariable(f"dev_{c}", 0) for c in campaigns}

# Lead growth and budget increase constraints (Refined Constraints)
alpha = 0.2  # Growth factor for leads
beta = 0.3  # Growth factor for budget

# Objective Function: Minimize total deviation from weighted budget allocation
model += lpSum(weights[c] * deviation[c] for c in campaigns)

# Constraints
for c in campaigns:
    # Weighted budget allocation based on duration
    weighted_budget = weights[c] * sum(budget.values())

    # Deviation constraints
    model += x[c] - weighted_budget <= deviation[c]
    model += weighted_budget - x[c] <= deviation[c]

    # Ensure optimized budget is within a reasonable range based on duration
    model += x[c] >= (current_spend[c] / max(1, duration[c])) + 1  # Scale spend per month, avoid division by zero
    model += x[c] <= current_spend[c] * (1 + beta * (duration[c] ** 0.5))  # Controlled increase

    # Ensure optimized leads follow a realistic growth pattern
    model += optimized_leads[c] >= actual_leads[c] + 1  # Optimized leads > actual leads
    model += optimized_leads[c] >= target_leads[c] * 0.9  # Allowing slight deviation from target
    model += optimized_leads[c] <= actual_leads[c] * (1 + alpha * (duration[c] ** 0.7))  # Exponential growth factor

    # Introduce a new variable for proportion (avoiding division)
    proportion = LpVariable(f"prop_{c}", 0, 1)
    model += proportion * budget[c] == x[c]  # Define proportion constraint

    # Ensure optimized leads are proportional to the optimized budget and duration
    model += optimized_leads[c] <= proportion * actual_leads[c] * (1 + 0.1 * duration[c])  # Proportional adjustment

# Constraint: Total optimized budget should be less than or equal to total actual budget
model += lpSum(x[c] for c in campaigns) <= sum(budget.values())

# Solve the model
model.solve()

# Print Results
print("\nOptimized Campaign Allocation:")
total_optimized_budget = 0
total_actual_budget = sum(budget.values())
total_actual_leads = sum(actual_leads.values())
total_optimized_leads = 0

# Store campaigns where optimized leads > actual leads by more than 1
significant_improvements = []

def get_var_value(var, default=0):
    return max(default, round(var.varValue) if var.varValue is not None else default)  # Ensure natural numbers for leads

for c in campaigns:
    optimized_budget = get_var_value(x[c])
    optimized_leads_val = get_var_value(optimized_leads[c], actual_leads[c] + 1)
    total_optimized_budget += optimized_budget
    total_optimized_leads += optimized_leads_val

    if optimized_leads_val - actual_leads[c] > 1:
        significant_improvements.append((c, optimized_budget, budget[c], current_spend[c], duration[c], actual_leads[c], optimized_leads_val, target_leads[c]))

    print(f"Campaign {c}: Optimized Budget = ${optimized_budget:.2f}, Actual Budget = ${budget[c]:.2f}, "
          f"Actual Spend = ${current_spend[c]:.2f}, Duration = {duration[c]} months, "
          f"Actual Leads = {actual_leads[c]}, Optimized Leads = {optimized_leads_val}, Target Leads = {target_leads[c]}")

# Print Total Summary
print("\nTotal Summary:")
print(f"Total Actual Budget: ${total_actual_budget:.2f}")
print(f"Total Optimized Budget: ${total_optimized_budget:.2f}")
print(f"Total Actual Leads: {total_actual_leads}")
print(f"Total Optimized Leads: {total_optimized_leads}")
print(f"Status: {LpStatus[model.status]}")

# Print campaigns where optimized leads increased significantly
print("\nCampaigns with Significant Lead Improvement (Optimized Leads - Actual Leads > 1):")
for c in significant_improvements:
    print(f"Campaign {c[0]}: Optimized Budget = ${c[1]:.2f}, Actual Budget = ${c[2]:.2f}, "
          f"Actual Spend = ${c[3]:.2f}, Duration = {c[4]} months, "
          f"Actual Leads = {c[5]}, Optimized Leads = {c[6]}, Target Leads = {c[7]}")




Optimized Campaign Allocation:
Campaign C1: Optimized Budget = $9202.00, Actual Budget = $9202.00, Actual Spend = $6995.00, Duration = 4 months, Actual Leads = 254, Optimized Leads = 255, Target Leads = 286
Campaign C2: Optimized Budget = $6831.00, Actual Budget = $6831.00, Actual Spend = $6126.00, Duration = 5 months, Actual Leads = 261, Optimized Leads = 262, Target Leads = 209
Campaign C3: Optimized Budget = $8952.00, Actual Budget = $9468.00, Actual Spend = $8677.00, Duration = 3 months, Actual Leads = 233, Optimized Leads = 234, Target Leads = 283
Campaign C4: Optimized Budget = $10008.00, Actual Budget = $10008.00, Actual Spend = $8651.00, Duration = 5 months, Actual Leads = 117, Optimized Leads = 118, Target Leads = 276
Campaign C5: Optimized Budget = $7920.00, Actual Budget = $7920.00, Actual Spend = $5734.00, Duration = 5 months, Actual Leads = 272, Optimized Leads = 273, Target Leads = 233
Campaign C6: Optimized Budget = $5968.00, Actual Budget = $11608.00, Actual Spend = $9

Optimized Budget Allocation for Maximizing Leads: A Campaign Performance Enhancement Model